In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/french-twitter-sentiment-analysis/french_tweets.csv


# Importation des bibliothéques nécessaires

In [2]:
import os
import re
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import torch
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks, optimizers

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

# Reproductibilité
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)
torch.manual_seed(SEED)

# Style
sns.set(style='whitegrid')

!pip install -q transformers torch gradio

2025-10-17 09:46:08.194402: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760694368.427606      37 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760694368.493824      37 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.6/68.6 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 82.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 84.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 12.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 84.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━

# Compréhension des données

In [3]:
# Charger les données
data = pd.read_csv('/kaggle/input/french-twitter-sentiment-analysis/french_tweets.csv')
print(f"Taille du dataset : {data.shape}")
data.head()

print("\nInfo :")
print(data.info())
print("\nDistribution originale des labels :")
print(data['label'].value_counts())

Taille du dataset : (1526724, 2)

Info :
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1526724 entries, 0 to 1526723
Data columns (total 2 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   label   1526724 non-null  int64 
 1   text    1526724 non-null  object
dtypes: int64(1), object(1)
memory usage: 23.3+ MB
None

Distribution originale des labels :
label
0    771604
1    755120
Name: count, dtype: int64


# Préparation des données

## Nettoyage des textes 

In [4]:
def clean_text(text):
    if pd.isna(text):
        return ""
    text = str(text).lower()
    text = re.sub(r'http\S+|www\.\S+', ' ', text)
    text = re.sub(r'@\w+', ' ', text)
    text = re.sub(r'#', ' ', text)
    text = re.sub(r'[^a-z0-9àâäéèêëïîôöùûüçœæ\-\s]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

data['text_clean'] = data['text'].astype(str).map(clean_text)

## Equilibrage des données

In [5]:
TARGET_PER_CLASS = 60000

def safe_sample_per_class(df, label_col='label', n=TARGET_PER_CLASS, random_state=SEED):
    parts = []
    for lbl, group in df.groupby(label_col):
        sample_n = min(len(group), n)
        parts.append(group.sample(n=sample_n, random_state=random_state))
    return pd.concat(parts).reset_index(drop=True)

data_balanced = safe_sample_per_class(data, label_col='label', n=TARGET_PER_CLASS)
print("Distribution équilibrée :")
print(data_balanced['label'].value_counts())

Distribution équilibrée :
label
0    60000
1    60000
Name: count, dtype: int64


## Chargemenr du modéle d'embedding et Tokenization

In [6]:
from transformers import AutoTokenizer, AutoModel

MODEL_NAME = "sentence-transformers/all-MiniLM-L12-v2"
print(f"Chargement du modèle MiniLM : {MODEL_NAME}")

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
model = AutoModel.from_pretrained(MODEL_NAME, trust_remote_code=True)

print(f"\nArchitecture du modèle MiniLM :")
print(f"- Couches: {model.config.num_hidden_layers}")
print(f"- Dimensions cachées: {model.config.hidden_size}")
print(f"- Têtes d'attention: {model.config.num_attention_heads}")

Chargement du modèle MiniLM : sentence-transformers/all-MiniLM-L12-v2


tokenizer_config.json:   0%|          | 0.00/352 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/133M [00:00<?, ?B/s]


Architecture du modèle MiniLM :
- Couches: 12
- Dimensions cachées: 384
- Têtes d'attention: 12


## Fonction de pooling + embeddings

In [7]:
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output.last_hidden_state
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask

def get_minilm_embeddings(texts, batch_size=32):
    model.eval()
    all_embeddings = []
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i+batch_size]
        encoded_input = tokenizer(
            batch_texts.tolist(),
            padding=True,
            truncation=True,
            max_length=128,
            return_tensors='pt'
        )
        with torch.no_grad():
            model_output = model(**encoded_input)
        embeddings = mean_pooling(model_output, encoded_input['attention_mask'])
        all_embeddings.extend(embeddings.numpy())
    return np.array(all_embeddings)

## Application des embeddings

In [ ]:
texts = data_balanced['text_clean']
labels = data_balanced['label']

minilm_embeddings = get_minilm_embeddings(texts)

In [9]:
test_texts = pd.Series([
    "J'adore ce produit !",
    "Je déteste ce service.",
    "C'était correct, sans plus."
])

emb_test = get_minilm_embeddings(test_texts)
print("Shape des embeddings :", emb_test.shape)
print(emb_test[0][:10])  # premières valeurs pour vérifier

Shape des embeddings : (3, 384)
[-0.22523013  0.5134914   0.187118   -0.3391628  -0.11511293 -0.0803717
  0.26003262  0.203393   -0.22086304 -0.02835667]


## Visualisation PCA et t-SNE

In [10]:
# PCA
pca = PCA()
pca.fit(minilm_embeddings)
plt.figure(figsize=(10,5))
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('Nombre de composantes PCA')
plt.ylabel('Variance expliquée cumulée')
plt.title('PCA sur les embeddings MiniLM')
plt.grid(True)
plt.show()

# t-SNE
le = LabelEncoder()
labels_encoded = le.fit_transform(labels)
tsne = TSNE(n_components=2, random_state=SEED, perplexity=30)
embeddings_2d = tsne.fit_transform(minilm_embeddings)

plt.figure(figsize=(12,8))
plt.scatter(embeddings_2d[:,0], embeddings_2d[:,1], c=labels_encoded, cmap='Set2', s=8, alpha=0.7)
plt.title("t-SNE - MiniLM embeddings")
plt.show()

NameError: name 'minilm_embeddings' is not defined

# Modélisation

## Entraînement du classificateur

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    minilm_embeddings, labels, test_size=0.2, random_state=SEED, stratify=labels
)

label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

num_classes = len(label_encoder.classes_)
input_dim = X_train.shape[1]

print(f"Train: {X_train.shape}, Test: {X_test.shape}")
print(f"Nombre de classes : {num_classes}")

## Réseau de neurones

In [ ]:
def build_minilm_classifier(input_dim, num_classes):
    model = models.Sequential([
        layers.Dense(256, activation='relu', input_shape=(input_dim,)),
        layers.BatchNormalization(),
        layers.Dropout(0.4),

        layers.Dense(128, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.3),

        layers.Dense(64, activation='relu'),
        layers.Dropout(0.2),

        layers.Dense(num_classes, activation='softmax')
    ])

    model.compile(
        optimizer=optimizers.Adam(0.001),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

classifier = build_minilm_classifier(input_dim, num_classes)
classifier.summary()

## Entraînement

In [ ]:
early_stop = callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

history = classifier.fit(
    X_train, y_train_encoded,
    validation_data=(X_test, y_test_encoded),
    epochs=10,
    batch_size=256,
    callbacks=[early_stop],
    verbose=1
)

# Evaluation

In [ ]:
y_pred = classifier.predict(X_test)
y_pred_labels = np.argmax(y_pred, axis=1)

print("Rapport de classification :")
print(classification_report(y_test_encoded, y_pred_labels, target_names=label_encoder.classes_))

cm = confusion_matrix(y_test_encoded, y_pred_labels)
plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_)
plt.title('Matrice de confusion')
plt.ylabel('Vrai label')
plt.xlabel('Label prédit')
plt.show()

# Déploiement

In [ ]:
import gradio as gr
import pandas as pd

# ⚠️ Ici, on suppose que `clean_text`, `tokenizer`, `model_minilm` et `classifier`
# sont déjà définis et entraînés dans les cellules précédentes.

# 🧠 Fonction pour prédire le sentiment d’un texte unique
def predict_sentiment(text):
    if not text.strip():
        return "⚠️ Texte vide"
    cleaned_text = clean_text(text)
    emb = get_minilm_embeddings(pd.Series([cleaned_text]))
    prediction = classifier.predict(emb)
    label_id = prediction.argmax(axis=1)[0]
    label = label_encoder.inverse_transform([label_id])[0]
    return label

# 📂 Fonction pour prédire le sentiment d’un fichier texte (ou CSV)
def predict_file(file):
    if file is None:
        return "Aucun fichier uploadé"

    # Lecture du fichier
    try:
        df = pd.read_csv(file.name, header=None, names=['text'])
    except Exception:
        with open(file.name, 'r', encoding='utf-8') as f:
            lines = f.readlines()
        df = pd.DataFrame(lines, columns=['text'])

    df['text_clean'] = df['text'].map(clean_text)
    embeddings = get_minilm_embeddings(df['text_clean'])
    preds = classifier.predict(embeddings)
    label_ids = preds.argmax(axis=1)
    df['sentiment'] = label_encoder.inverse_transform(label_ids)

    return df[['text', 'sentiment']]

# 🎨 Interface Gradio
with gr.Blocks() as demo:
    gr.Markdown("## 🧠 Analyse de Sentiment des Tweets en Français")
    gr.Markdown("Prédisez si un tweet ou un fichier contient des sentiments **positifs** ou **négatifs**.")

    with gr.Tab("📌 Prédiction par texte"):
        text_input = gr.Textbox(label="Entrez un texte ou un tweet", placeholder="Ex: J'adore ce produit !")
        output_label = gr.Label(label="Sentiment")
        btn1 = gr.Button("Prédire")
        btn1.click(fn=predict_sentiment, inputs=text_input, outputs=output_label)

    with gr.Tab("📂 Prédiction par fichier"):
        file_input = gr.File(label="Uploader un fichier TXT ou CSV")
        output_dataframe = gr.Dataframe(headers=["Texte", "Sentiment"])
        btn2 = gr.Button("Analyser le fichier")
        btn2.click(fn=predict_file, inputs=file_input, outputs=output_dataframe)

demo.launch()